In [7]:
import requests
import urllib.parse
import xml
import time
import re
import copy

import math
import numpy as np
import pandas as pd
import tensorflow as tf


from requests import exceptions
from bs4 import BeautifulSoup
import bs4
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
WAIT_TIME = 2

def get_CountryNameInColumn_At_WRC_ChampionshipStanding(Selenium_driver,target_url):
    result = []
    Selenium_driver.get(target_url)
    time.sleep(WAIT_TIME)
    heading = Selenium_driver.find_element(By.TAG_NAME, "main")
    thead =  heading.find_element(By.TAG_NAME,"thead")

    img_elements = thead.find_elements(By.TAG_NAME,"img")

    for img_element in img_elements:
        alt_value = img_element.get_attribute("alt")
        result.append(alt_value)
    return result

def get_index_of_country_Alltime(Country_Dictionary_ForYear):
    index= []
    for year in Country_Dictionary_ForYear.keys():
        for country_name in Country_Dictionary_ForYear[year]:
            if country_name not in index:
                index.append(country_name)
    return index

def get_Country_WRC_held_Alltime(Selenium_driver,held_years_list,WRC_ChampionshipStanding_url):
    Country_WRC_held = {}
    for year in held_years_list:
        target_url = WRC_ChampionshipStanding_url + f'/season-{year}/wrc/'
        time.sleep(WAIT_TIME)
        tmp_country = get_CountryNameInColumn_At_WRC_ChampionshipStanding(Selenium_driver,target_url)
        Country_WRC_held[year] = tmp_country
    return Country_WRC_held

def Algorithm_searching_common_country(Country_Dictionary_ForYear):
    index= []
    total_items = []
    result = []
    for year in Country_Dictionary_ForYear.keys():
        for country_name in Country_Dictionary_ForYear[year]:
            if country_name not in index:
                index.append(country_name)
            total_items.append(country_name)
    thshold = len(Country_Dictionary_ForYear.keys())
    for country in index:
        print(thshold)
        print(total_items.count(country))
        if total_items.count(country) >= thshold:
            result.append(country)
    
    return result

def Search_Common_Country(Selenium_driver,held_years_list,WRC_ChampionshipStanding_url):
    all_Country_WRC_held = get_Country_WRC_held_Alltime(Selenium_driver,held_years_list,WRC_ChampionshipStanding_url)
    return Algorithm_searching_common_country(all_Country_WRC_held)

def get_TableRow_At_WRC_ChampionshipStanding_p1(Selenium_driver,target_url):
    result_dict={}
    Selenium_driver.get(target_url)
    time.sleep(WAIT_TIME)
    table = Selenium_driver.find_element(By.TAG_NAME,"table")
    table_rows = table.find_elements(By.TAG_NAME, "tr")

    for row in table_rows:
        cells = row.find_elements(By.TAG_NAME, "td")
        row_data = [cell.text for cell in cells]
        if len(row_data)<1:
            continue
        else:
            result_dict[int(row_data[0])] = row_data

    return result_dict

def get_TableRow_At_WRC_ChampionshipStanding_p2(Selenium_driver,target_url):
    result_dict={}
    Selenium_driver.get(target_url)
    time.sleep(WAIT_TIME)
    table = Selenium_driver.find_element(By.TAG_NAME,"table")
    table_rows = table.find_elements(By.TAG_NAME, "tr")

    for row in table_rows:
        cells = row.find_elements(By.TAG_NAME, "td")
        row_data = [cell.text for cell in cells]
        if len(row_data)<1:
            continue
        else:
            row_data.remove('')
            result_dict[int(row_data[0])] = row_data

    return result_dict


def get_Table_Of_WRC_ChampionshipStanding_Alltime(Selenium_driver,period1,period2,WRC_ChampionshipStanding_url):
    table_hampionshipStanding_Alltime= {}
    for year in period1:
        target_url = WRC_ChampionshipStanding_url + f'/season-{year}/wrc/'
        time.sleep(WAIT_TIME)
        tmp_standing = get_TableRow_At_WRC_ChampionshipStanding_p1(Selenium_driver,target_url)
        table_hampionshipStanding_Alltime[year] = tmp_standing

    for year in period2:
        target_url = WRC_ChampionshipStanding_url + f'/season-{year}/wrc/'
        time.sleep(WAIT_TIME)
        tmp_standing = get_TableRow_At_WRC_ChampionshipStanding_p2(Selenium_driver,target_url)
        table_hampionshipStanding_Alltime[year] = tmp_standing

    return table_hampionshipStanding_Alltime

def extract_ranking(data_lst):
    new_list = []
    for item in data_lst:
        tmp_item = item.split('\n')
        tmp_value = 0
        # print(item,len(tmp_item))
        if ('D' in item)|('R' in item)|(len(tmp_item)<=1):
            tmp_value = 100
        else:
            tmp_value = re.sub('[.()-]','',tmp_item[1])
            if len(tmp_value)<1:
                tmp_value = 100
         
        new_list.append(int(tmp_value))
    return new_list

def RoughTable_processing(table):
    table_dict = {}    
    for year in table:
        # print(year)
        # if year !=2022:
        #     continue
        data_year = table[year]
        copy_data = copy.deepcopy(data_year)
        for championship_rank in copy_data:
            record = copy_data[championship_rank]
            
            record_year = year
            record_champ_rank = int(record[0])
            record_racer_name = record[1]
            record_racer_TotalScore = int(record[-1])
            record.remove(record[0])
            record.remove(record[0])
            record.pop()
            # print(record)
            # print(year)
            result_rank = extract_ranking(record)
            # print(result_rank)

            table_dict[record_year,record_champ_rank] = {'championship_rank':record_champ_rank,
                                        'Driver_name':record_racer_name,
                                        'Total_Score':record_racer_TotalScore,
                                        'rank_list':result_rank
                                    }
    return table_dict



In [8]:
### main

### Selenium
driver_dir = '/chromedriver_win32'
chrome_options = Options()
chrome_options.add_argument("--headless") 
service = Service(driver_dir)  
driver = webdriver.Chrome(service=service, options=chrome_options)
Years_WRC_held = list(range(2001,2023))
period1 = list(range(2001,2019))
period2 = list(range(2019,2023))
WRC_ChampionshipStanding_url = "https://www.wrc.com/en/results-standings/championship-standings/seasons/championship-standings"

# Country_WRC_held_alltime = get_Country_WRC_held_Alltime(driver,Years_WRC_held,WRC_ChampionshipStanding_url)
# country_index = get_index_of_country_Alltime(Country_WRC_held_alltime)

Table = get_Table_Of_WRC_ChampionshipStanding_Alltime(driver,period1,period2,WRC_ChampionshipStanding_url)

# Close the Selenium driver
driver.quit()


In [9]:
### main2
soft_table = RoughTable_processing(Table)
soft_table

{}

In [10]:
import psycopg2

## Database Insertion

# Establish a connection to the PostgreSQL database
conn = psycopg2.connect(
    host="localhost",  # Replace with your host
    database="wrc",  # Replace with your database name
    user="postgres",  # Replace with your username
    password="ghkanf0810!"  # Replace with your password
)

cursor = conn.cursor()

create_table_query = """CREATE TABLE IF NOT EXISTS rally_results (
    year INT,
    position INT,
    championship_rank INT,
    driver_name VARCHAR(100),
    total_score INT,
    rank_list INT[]
);
"""

cursor.execute(create_table_query)


for key, values in soft_table.items():
    query = """
    INSERT INTO rally_results (year, position, championship_rank, driver_name, total_score, rank_list)
    VALUES (%s, %s, %s, %s, %s, %s)
    """
    year, position = key
    championship_rank = values['championship_rank']
    driver_name = values['Driver_name']
    total_score = values['Total_Score']
    rank_list = values['rank_list']

    cursor.execute(query, (year, position, championship_rank, driver_name, total_score, rank_list))

# Commit the changes to the database
conn.commit()

# Close the cursor and connection
cursor.close()
conn.close()

In [11]:
## get data from database

conn = psycopg2.connect(
    host="localhost",  # Replace with your host
    database="wrc",  # Replace with your database name
    user="postgres",  # Replace with your username
    password="ghkanf0810!"  # Replace with your password
)

cursor = conn.cursor()

select_query = "SELECT championship_rank , rank_list FROM rally_results"

cursor.execute(select_query)

# Fetch all the rows returned by the query
rows = cursor.fetchall()

mean_lst = []
std_lst = []
stage_num_lst = []
rank_lst = []
best_rank_lst = []

# Process the fetched rows
for row in rows:
    mean_lst.append(sum(row[1])/len(row[1]))
    std_lst.append(np.std(row[1]))
    stage_num_lst.append(len(row[1]))
    rank_lst.append(row[0])
    best_rank_lst.append(min(row[1]))
    print(row[0])

cursor.close()
conn.close()


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
17
16
18
19
20
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
1

In [12]:
# ## export data
# import subprocess

# # Define the database connection details
# host = "localhost"
# port = "5432"
# database = "wrc"
# username = "postgres"
# password = "ghkanf0810!"

# # Define the output file path
# output_file = "wrc_db.sql"

# # Construct the pg_dump command
# command = [
#     "pg_dump",
#     f"--host={host}",
#     f"--port={port}",
#     f"--dbname={database}",
#     f"--username={username}",
#     f"--file={output_file}",
# ]

# # Add the password if provided
# if password:
#     command.append(f"--password={password}")

# # Execute the pg_dump command
# subprocess.run(command)

In [6]:
data = {
    'Mean': mean_lst,
    'Standard Deviation': std_lst,
    'Stage Number': stage_num_lst,
    'Rank': rank_lst,
    'Best Rank': best_rank_lst
}

df = pd.DataFrame(data)
df.info()
df.to_csv("wrc_db.csv")


df.loc[df['Rank']<4,'Rank'] = 1
df.loc[df['Rank']>3,'Rank'] = 0

X = df.drop('Rank',axis = 1,inplace= False)
y = df['Rank']


NameError: name 'mean_lst' is not defined

In [96]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Input
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42 , stratify=y)

model = Sequential() 
model.add(Input((4,)))
model.add(Dense(10, activation='relu'))
model.add(Dense(1, activation='sigmoid')) # 출력층
model.compile(optimizer='adam', # 옵티마이저
              loss='binary_crossentropy', # 손실함수
              metrics=['accuracy']) # 학습 지표
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 10)                50        
                                                                 
 dense_5 (Dense)             (None, 1)                 11        
                                                                 
Total params: 61
Trainable params: 61
Non-trainable params: 0
_________________________________________________________________


In [123]:
model.fit(X_train, y_train, epochs=10)
model.evaluate(X_test,  y_test, verbose=2)

Epoch 1/10
12/12 [==============================] - 0s 1ms/step - loss: 0.1106 - accuracy: 0.9569
Epoch 2/10
12/12 [==============================] - 0s 2ms/step - loss: 0.1099 - accuracy: 0.9596
Epoch 3/10
12/12 [==============================] - 0s 2ms/step - loss: 0.1089 - accuracy: 0.9569
Epoch 4/10
12/12 [==============================] - 0s 1ms/step - loss: 0.1094 - accuracy: 0.9569
Epoch 5/10
12/12 [==============================] - 0s 1ms/step - loss: 0.1094 - accuracy: 0.9569
Epoch 6/10
12/12 [==============================] - 0s 1ms/step - loss: 0.1091 - accuracy: 0.9569
Epoch 7/10
12/12 [==============================] - 0s 1ms/step - loss: 0.1100 - accuracy: 0.9596
Epoch 8/10
12/12 [==============================] - 0s 1ms/step - loss: 0.1092 - accuracy: 0.9542
Epoch 9/10
12/12 [==============================] - 0s 1ms/step - loss: 0.1087 - accuracy: 0.9569
Epoch 10/10
12/12 [==============================] - 0s 1ms/step - loss: 0.1090 - accuracy: 0.9569
6/6 - 0s - loss: 0.

[0.11766047030687332, 0.9453551769256592]

In [131]:
import pickle

with open('model.pkl', 'wb') as file:
    pickle.dump(model, file)


In [4]:
import psycopg2

## Database Insertion

# Establish a connection to the PostgreSQL database
conn = psycopg2.connect(
    host="localhost",  # Replace with your host
    database="wrc",  # Replace with your database name
    user="postgres",  # Replace with your username
    password="ghkanf0810!"  # Replace with your password
)

cursor = conn.cursor()
